<a href="https://colab.research.google.com/github/forouzanfarahani/Pathway-Interaction-/blob/main/Pathway_Interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run this code to install the package. 
# installing a package for reading .abf files
%pip install --upgrade pyabf

     |████████████████████████████████| 51kB 3.0MB/s 
  Created wheel for pyabf: filename=pyabf-2.2.8-cp36-none-any.whl size=47650 sha256=b32a7a7db1e84ad71591699e1ee15f30146474919f743e70989c00e93ae9b171
  Stored in directory: /root/.cache/pip/wheels/d6/f5/00/be17ca092e5e11c1a65af2424ebb3bf7040ab3bfb49e6e17f7
Successfully built pyabf


In [ ]:
from scipy.optimize import curve_fit
import numpy as np
import pyabf
import os
import warnings
warnings.filterwarnings("ignore")
import glob
import pandas as pd

In [ ]:
#@title import matplotlib and set defaults
from plotly import __version__
%matplotlib inline

import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 


cf.go_offline()
init_notebook_mode(connected=False)

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
#@title
def artifcat_removal(x_in, start, end):
  x_out = x_in.copy()
  from scipy.interpolate import interp1d
  xq = np.arange(start,end)
  f =  interp1d(np.array([start,end]),x_in[[start, end]],kind='linear')
  x_out[xq] = x_out[xq] - f(xq)
  return x_out


def manual_manip_cathodal(signal_in, address):
  f = 20000
  x_in = signal_in.copy()
  # if address== '/content/drive/My Drive/Pathway Interaction Project/Raw Data/21107003.abf' :
  #   start = f*1.02
  #   end = f*1.57
  #   signal_ = artifcat_removal(x_in, start, end)    
  #   start = f*1.57
  #   end = f*1.99
  #   signal_out = artifcat_removal(signal_, start, end)
  start = f*1.02
  end = f*1.57
  signal_ = artifcat_removal(x_in, start, end)    
  start = f*1.57
  end = f*1.99
  signal_out = artifcat_removal(signal_, start, end)
  return signal_out

def manual_manip_anodal(signal_in, address):
  f = 20000
  x_in = signal_in.copy()
  start = f*1.02
  end = f*1.99
  signal_out = artifcat_removal(x_in, start, end)
  return signal_out

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:
  
ProjectFolderName = 'Pathway Interaction Project'

#don't edit these:
data_path = '/content/drive/My Drive/Pathway Interaction Project/Raw Data/' #Enter the list of videos or folder to analyze.
#!ls "/content/drive/My Drive/Pathway Interaction Project/Raw Data/"

In [ ]:
# getting list of 
data_files = glob.glob("/content/drive/My Drive/Pathway Interaction Project/Raw Data/*.abf")

In [ ]:
abf = pyabf.ABF(data_files[-95])
df = pd.DataFrame({
    'time': abf.sweepX, 
    'voltage': abf.sweepY
}
)

In [ ]:
configure_plotly_browser_state()

idx = []
EPSP_slope = {'control': [], 'anodal': [], 'cathodal': []}

for sweepNumber in abf.sweepList[0:2]:   
    abf.setSweep(sweepNumber)
    df.iplot(kind='line',x='time',y='voltage', color=['white'], 
theme='solar', mode='lines',title='membrane potential')
print(abf.sweepList[-1])

In [ ]:
print(abf.headerText) # display header information in the console
abf.headerLaunch() # display header information in a web browser

In [ ]:
address_book = pd.read_excel('/content/drive/My Drive/Pathway Interaction Project/data_list.xlsx')
path_keys ={'TBS-SC','TBS-EC'}
address =  {}
for path_key in path_keys:
  address[path_key] = address_book[path_key]
  

In [ ]:


abf = pyabf.ABF(address['TBS-EC'][15])
df = pd.DataFrame({
    'time': abf.sweepX,    
}
)
sweeps = []
for sweepNumber in abf.sweepList:
    abf.setSweep(sweepNumber)
    df['sweep_'+str(sweepNumber)] = abf.sweepY
    sweeps.append('sweep_'+str(sweepNumber))


ValueError: ignored

In [ ]:
configure_plotly_browser_state()

idx = []
EPSP_slope = {'control': [], 'anodal': [], 'cathodal': []}

  
df.iplot(kind='line',x='time',y=['sweep_1','sweep_5',], color=['white'], theme='solar', mode='lines',title='membrane potential')
    